In [ ]:
import pandas as pd

# Definir las rutas de los archivos
ruta_entrada = r'C:\Users\Victor B\Desktop\PROJECTO\orders.csv'
ruta_salida = r'C:\Users\Victor B\Desktop\PROJECTO\orders_ok.csv'

# Leer el archivo CSV
try:
    df = pd.read_csv(ruta_entrada)
except FileNotFoundError:
    print(f"Archivo no encontrado: {ruta_entrada}")
    exit(1)
except pd.errors.ParserError as e:
    print(f"Error al analizar el CSV: {e}")
    exit(1)

# Asegurarse de que la columna 'Purchase Date' esté en el formato correcto
try:
    df['Purchase Date'] = pd.to_datetime(df['Purchase Date'], format='%b %d, %Y %I:%M:%S %p')
except ValueError as e:
    print(f"Error al convertir las fechas: {e}")
    exit(1)

# Extraer la fecha y la hora en nuevas columnas
df['fecha'] = df['Purchase Date'].dt.date
df['DIA'] = df['Purchase Date'].dt.day
df['MES'] = df['Purchase Date'].dt.month
df['MES NOM'] = df['Purchase Date'].dt.strftime('%B')  # Nombre completo del mes
df['AÑO'] = df['Purchase Date'].dt.year
df['HORA'] = df['Purchase Date'].dt.hour

# Asegurarse de que las columnas estén en formato numérico
columnas_a_convertir = [
    'Grand Total (Base)', 
    'Grand Total (Purchased)', 
    'Cart Subtotal', 
    'Shipping and Handling', 
    'Total Refunded'
]

for columna in columnas_a_convertir:
    if columna in df.columns:
        df[columna] = pd.to_numeric(df[columna], errors='coerce')

# Crear nuevas columnas para la ciudad y el código postal a partir de 'Billing Address'
def extraer_ciudad_y_codigo_postal(direccion):
    partes = direccion.split(',')
    if len(partes) >= 3:
        ciudad = partes[-3].strip()
        codigo_postal = partes[-1].strip()
        return pd.Series([ciudad, codigo_postal])
    else:
        return pd.Series([None, None])

df[['Ciudad', 'Código Postal']] = df['Billing Address'].apply(extraer_ciudad_y_codigo_postal)

# Calcular el descuento aplicado en porcentaje, incluyendo gastos de envío
def calcular_descuento(row):
    try:
        total_con_envio = row['Grand Total (Purchased)'] + row['Shipping and Handling']
        if total_con_envio == 0:
            return 0
        descuento = ((total_con_envio - row['Cart Subtotal']) / total_con_envio) * 100
        # Asegurar que el descuento no sea negativo
        if descuento < 0:
            return 0
        return descuento
    except (TypeError, ZeroDivisionError):
        return 0

df['Descuento (%)'] = df.apply(calcular_descuento, axis=1)

# Calcular el descuento medio por día
descuento_medio_por_dia = df.groupby('fecha')['Descuento (%)'].mean().reset_index()
descuento_medio_por_dia.rename(columns={'Descuento (%)': 'Descuento Medio (%)'}, inplace=True)

# Unir el descuento diario promedio al DataFrame
df = df.merge(descuento_medio_por_dia, on='fecha', how='left')

# Calcular el precio medio del ticket por día
precio_medio_por_dia = df.groupby('fecha')['Cart Subtotal'].mean().reset_index()
precio_medio_por_dia.rename(columns={'Cart Subtotal': 'Precio Medio Ticket'}, inplace=True)

# Unir el DataFrame original con el DataFrame de precios medios
df = df.merge(precio_medio_por_dia, on='fecha', how='left')

# Formatear los valores numéricos para el archivo CSV de salida
columnas_a_formatear = columnas_a_convertir + ['Precio Medio Ticket']
for columna in columnas_a_formatear:
    if columna in df.columns:
        df[columna] = df[columna].apply(lambda x: f"{x:.2f}".replace('.', ','))

# Formatear los valores de porcentaje para las columnas de descuentos
df['Descuento (%)'] = df['Descuento (%)'].apply(lambda x: f"{x:.2f}%".replace('.', ','))
df['Descuento Medio (%)'] = df['Descuento Medio (%)'].apply(lambda x: f"{x:.2f}%".replace('.', ','))

# Guardar el DataFrame modificado en un nuevo archivo CSV
df.to_csv(ruta_salida, index=False)

print('Archivo CSV modificado y guardado como orders_ok.csv')


In [ ]:
# Codigos Postales y Municipios Asociados
# Fuente: Callejero del Censo Electoral (INE)
# URL: https://github.com/inigoflores/ds-codigos-postales-ine-es?tab=readme-ov-file 
# Tipo: Texto de ancho fijo comprimido (.zip)
# Datos procesados: /data/codigos_postales_municipios.csv

# Población
# Fuente: (INE)
# URL: https://www.ine.es/dynt3/inebase/index.htm?padre=525 
# Tipo: Texto de ancho fijo comprimido (.zip)
# Datos procesados: /data/pobmun23.csv

# Unir con la variable común MUNICIPIO_ID